## Packages

In [1]:
from pylab import *
import pretty_midi
import librosa          
import librosa.display     
import IPython.display     
import matplotlib.pyplot as plt 
import matplotlib.style as ms   
import numpy as np          
ms.use('seaborn-muted')         
from __future__ import print_function

from sklearn.naive_bayes import GaussianNB
import numpy as np
import pandas as pd
from pandas import DataFrame

from pylab import *
%matplotlib inline 
import random
import copy

## Some Functions

### Handy

In [10]:
def matrix2notes(m):
    #input: m, a N by 4 matrix (4 means starting time, ending time, pitch, velocity)
    # first sort the matrix to make sure starting time is in order
    m = array(m)
    m = m[argsort(m[:,0]),:] #argsort gives you the index before the sorting
    # transfer the databack to note list
    notes=[pretty_midi.Note(start=m[i,0], end=m[i,1], pitch=int(m[i,2]), velocity= int(m[i,3]) ) 
                            for i in range(size(m,0))]
    return notes

sr = 22050

def conc(*args):
    chunk = array([])
    for i in args:
        chunk = append(chunk, i)
    return chunk

def am(plate, stuff):
    delta = len(stuff) - len(plate)
    if delta < 0:
        return plate * append(stuff, zeros(-delta))
    else:
        return plate * stuff[:len(plate)]

def save(name,wave,sr = 22050):
    librosa.output.write_wav(name,wave,sr)
    
def play(y, sr = sr):
    return IPython.display.Audio(data=y, rate=sr)

def create_note_matrix(start, end, note, vel):
    m = matrix(zeros((len(note), 4)))
    l = [start, end, note, vel]
    for i in range(4):
        if type(l[i]) != type([]) and type(l[i]) != type(np.array([])):
            m[:,i] = [l[i]]
        else:
            m[:,i] = matrix(l[i]).T
    return m

### Human Voice

In [4]:
def gus(tone, T = .3):
    gmin1 =  [cos((2*pi*tone(n/T)*T) + 3.4 * cos(2*pi*tone(n/T)*T/4)) * (sin(12*pi*n) + 18) for n in arange(0., T, 1./sr)]
    gmin2 =  [cos((2*pi*tone(n/T)*T*4) + 9 * cos(2*pi*tone(n/T)*T/4)) * (sin(12*pi*n) + 2) for n in arange(0., T, 1./sr)]
    fade_in = 1/16
    fade_out = 1/8
    head = exp(linspace(-5., 0., int(fade_in * T * sr)))
    mid = ones(int(T * sr * (1 - fade_in - fade_out)))
    tail = exp(linspace(0., -5., int(fade_out * T * sr)))
    envelope = conc(head, mid, tail)
    return am(add(gmin1, gmin2), envelope)

def woman(tone, T = .3):
    gmin1 =  [cos((8*pi*tone(n/T)*T) + 3.4 * cos(8*pi*tone(n/T)*T/4)) * (sin(12*pi*n) +
                                            17) for n in arange(0., T, 1./sr)]
    gmin2 =  [cos((8*pi*tone(n/T)*T*4) + 9 * cos(8*pi*tone(n/T)*T/4)) * (sin(12*pi*n) +
                                        2) for n in arange(0., T, 1./sr)]
    fade_in = 1/16
    fade_out = 1/8
    head = exp(linspace(-5., 0., int(fade_in * T * sr)))
    mid = ones(int(T * sr * (1 - fade_in - fade_out)))
    tail = exp(linspace(0., -5., int(fade_out * T * sr)))
    envelope = conc(head, mid, tail)
    return am(add(gmin1, gmin2), envelope)

def girl(tone, T = .3):
    gmin1 =  [cos((10*pi*tone(n/T)*T) + 3.4 * cos(8*pi*tone(n/T)*T/4)) * (sin(24*pi*n) +
                                            15) for n in arange(0., T, 1./sr)]
    gmin2 =  [cos((10*pi*tone(n/T)*T*4) + 9 * cos(8*pi*tone(n/T)*T/4)) * (sin(24*pi*n) +
                                        2) for n in arange(0., T, 1./sr)]
    fade_in = 1/16
    fade_out = 1/8
    head = exp(linspace(-5., 0., int(fade_in * T * sr)))
    mid = ones(int(T * sr * (1 - fade_in - fade_out)))
    tail = exp(linspace(0., -5., int(fade_out * T * sr)))
    envelope = conc(head, mid, tail)
    return am(add(gmin1, gmin2), envelope)

def boy(tone, T = .3):
    gmin1 =  [cos((4*pi*tone(n/T)*T) + 1.4 * cos(4*pi*tone(n/T)*T/4)) * (sin(12*pi*n) +
                                            13) for n in arange(0., T, 1./sr)]
    gmin2 =  [cos((4*pi*tone(n/T)*T*4) + 13 * cos(4*pi*tone(n/T)*T/4)) * (sin(12*pi*n) +
                                        2) for n in arange(0., T, 1./sr)]
    fade_in = 1/16
    fade_out = 1/8
    head = exp(linspace(-5., 0., int(fade_in * T * sr)))
    mid = ones(int(T * sr * (1 - fade_in - fade_out)))
    tail = exp(linspace(0., -5., int(fade_out * T * sr)))
    envelope = conc(head, mid, tail)
    return am(add(gmin1, gmin2), envelope)

def tone1(x):
    return x*688

def tone2(x):
    sep = .5
    start = 488
    end = 688
    a = (end - start) / sep / 2
    if x < sep:
        return a * x**2 + start * x
    else:
        return (x-sep) * end + a * sep**2 + start * sep

def tone2_high(x):
    sep = .5
    start = 488
    end = 602
    a = (end - start) / sep / 2
    if x < sep:
        return a * x**2 + start * x
    else:
        return (x-sep) * end + a * sep**2 + start * sep

def tone2_(x):
    return x * 602

def tone4(x):
    a = -200
    b = 760
    return a * x**2 + b * x

def tone3_(x):
    start = 265
    end = 396
    return (end - start)/2 * x**2 + start * x

def tone3(x):
    sep = .8
    start = 265
    end = 396
    if x < sep:
        return x * start
    else:
        return (x-sep) * end + sep * start

## Classifier -- Naiive Bayes

### Set Up Dataset

In [ ]:
energy=[]
for i in range(1,121):
    y, sr = librosa.load('Data/MP3/'+str(i)+'.mp3')
    librosa.feature.rmse(y=y)
    energy.append(np.average(librosa.feature.rmse(y=y)))
print(energy)

tempo_li=[]
for i in range(1,121):
    y, sr = librosa.load('Data/MP3/'+str(i)+'.mp3')
    onset_env = librosa.onset.onset_strength(y, sr=sr)
    tempo = librosa.beat.tempo(onset_envelope=onset_env, sr=sr)
    v=tempo[0]
    tempo_li.append(v)
print(i,tempo_li)

In [ ]:
file2 = 'Data/Metadata.csv'
#Metadata was manually created for 120 pieces of music

user = pd.read_csv(file2, sep=',', encoding='latin-1',skiprows=0)
temp=[]
for row in user.iterrows():
    index, data = row
    temp.append(data.tolist())
print(temp)
temp2 = copy.deepcopy(temp)

### Energy-based Prediction Model

In [ ]:
val2=[]
value=[]
used=[]
use=copy.deepcopy(used)
for i in range (0,83):
    num=random.sample(range(1, 120), 1)
    used.append(num[0])
    val2.append(temp[num[0]][2])
    value.append([energy[num[0]]])
print(num)
X = np.array(value)
Y = np.array(val2)
X.reshape(1,-1)

clf = GaussianNB()
clf.fit(X, Y)
use=copy.deepcopy(used)
print(clf.predict([[0.00066]]))
print(used)

In [ ]:
total=0
accurate=0
v=0
generated_energy=[]
test=[]
for i in range(84,120):
    num2=random.sample(range(1, 120), 1)
    if num2[0] not in used:
        test.append(num2[0])
        val=clf.predict([[energy[num2[0]]]])
        if ([temp2[num2[0]-1][2]]==clf.predict([[energy[num2[0]]]])):
            accurate+=1
            
        total+=1
        if((['negative'])==[temp2[num2[0]-1][2]]):
            v+=1
    generated_energy.append(val.tolist())
   

result2=(v/total)*100
result=(accurate/total)*100
print(generated_energy)
print('Prediction is '+str(result)+'% accurate.')
print('Baseline is ' +str(result2)+ '%')

### Tempo-based Prediction Model

In [ ]:
used=[]
val=[]
tempolst=[]
tempo_value=[]
print(used)
for i in range (0,83):
    val.append(temp[use[i]][1])
    tempo_value.append([tempo_li[use[i]]])

X = np.array(tempo_value)
Y = np.array(val)
X.reshape(1,-1)

clf = GaussianNB()
clf.fit(X, Y)

print(clf.predict([[100]]))

In [ ]:
total=0
accurate=0
v=0
generated_tempo=[]
for i in range(len(test)):
    vv=clf.predict([[tempo_li[test[i]]]])
    if ([temp2[test[i]-1][1]]==clf.predict([[tempo_li[test[i]]]])):
        accurate+=1
    total+=1
    if((['calm'])==[temp2[test[i]-1][1]]):
        v+=1
    generated_tempo.append(vv.tolist())
print(len(test))
print(len(generated_tempo))
print(len(generated_energy))
result2=(v/total)*100
result=(accurate/total)*100
print('Prediction is '+str(result)+'% accurate.')
print('Baseline is ' +str(result2)+ '%')

### Mood Prediction Model

In [ ]:
#Combining the predictions of the tempo-based model, which returned the labels "calm" and "energetic", with the energy-based
#model, which returned the labels "positive" and "negative", we obtain our final mood prediction and check its accuracy.

acc=0
tots=0
base=0
for i in range(len(test)):
    print(generated_tempo[i],generated_energy[i],[temp2[test[i]-1][3]])
    if generated_tempo[i]== ['calm'] and generated_energy[i]==['positive']:
        if [temp2[test[i]-1][3]]== ['calm']:
            acc+=1
    elif generated_tempo[i]== ['calm'] and generated_energy[i]==['negative']:
        if [temp2[test[i]-1][3]]== ['sad']:
            acc+=1
    elif generated_tempo[i]== ['energetic'] and generated_energy[i]==['positive']:
        if [temp2[test[i]-1][3]]== ['happy']:
            acc+=1
            base+=1
    elif generated_tempo[i]== ['energetic'] and generated_energy[i]==['negative']:
        if [temp2[test[i]-1][3]]== ['angry']:
            acc+=1
    tots+=1
b_res=base/tots*100
res=acc/tots*100
print('Prediction '+ str(res)+'% accurate.')
print('Baseline ' + str(b_res)+'%')

## Generator -- Markov Chain

### First Order Markov Chain

In [5]:
def markov(filename):
    midi_data = pretty_midi.PrettyMIDI(filename)
    l = []
    for i in midi_data.instruments:
        if not i.is_drum:
            pitches = [n.pitch for n in i.notes]
            average_pitch = sum(pitches)/len(pitches)
            l.append(average_pitch)
        else:
            l.append(0)
    index = l.index(max(l))
    if midi_data.instruments[index].is_drum:
        print("MAIN MELODY NOT IDENTIFIED")
        return [], [], [], [], [], []
    
    notes = midi_data.instruments[index].notes
    pitches = [n.pitch for n in notes]
    velocities = [n.velocity for n in notes]
    durs = [(n.end - n.start) for n in notes]
    
    pitch_id = list(set(pitches))
    pitch_id.sort()
    
    dur_id = list(set(durs))
    dur_id.sort()
    
    vel_id = list(set(velocities))
    vel_id.sort()
    
    Trans_pitch = np.zeros((len(pitch_id),len(pitch_id)))
    Trans_dur = np.zeros((len(dur_id),len(dur_id)))
    Trans_vel = np.zeros((len(vel_id),len(vel_id)))
    
    for i in range(len(notes)-1):
        if pitches[i] in pitch_id and pitches[i+1] in pitch_id:
            p1 = pitch_id.index(pitches[i])
            p2 = pitch_id.index(pitches[i+1])
            Trans_pitch[p1][p2] = Trans_pitch[p1][p2] + 1
            
        if durs[i] in dur_id and durs[i+1] in dur_id:
            d1 = dur_id.index(durs[i])
            d2 = dur_id.index(durs[i+1])
            Trans_dur[d1,d2] = Trans_dur[d1,d2] + 1
            
        if velocities[i] in vel_id and velocities[i+1] in vel_id:
            v1 = vel_id.index(velocities[i])
            v2 = vel_id.index(velocities[i+1])
            Trans_vel[v1,v2] = Trans_vel[v1,v1] + 1
            
    # normalization
    row_sums = Trans_pitch.sum(axis=1)
    Trans_pitch = Trans_pitch / row_sums[:, np.newaxis]

    row_sums = Trans_dur.sum(axis=1)
    Trans_dur = Trans_dur / row_sums[:, np.newaxis]
    
    row_sums = Trans_vel.sum(axis=1)
    Trans_vel = Trans_vel / row_sums[:, np.newaxis]

    return Trans_pitch, Trans_dur, Trans_vel, dur_id, pitch_id, vel_id

### Second Order Markov Chain

In [6]:
def markov2(filename):
    midi_data = pretty_midi.PrettyMIDI(filename)
    l = []
    for i in midi_data.instruments:
        if not i.is_drum:
            pitches = [n.pitch for n in i.notes]
            average_pitch = sum(pitches)/len(pitches)
            l.append(average_pitch)
        else:
            l.append(0)
    index = l.index(max(l))
    if midi_data.instruments[index].is_drum:
        print("MAIN MELODY NOT IDENTIFIED")
        return [], [], [], [], [], []
    
    notes = midi_data.instruments[index].notes
    pitches = [n.pitch for n in notes]
    velocities = [n.velocity for n in notes]
    durs = [(n.end - n.start) for n in notes]
    
    pitch_id = list(set(pitches))
    pitch_id.sort()
    
    vel_id = list(set(velocities))
    vel_id.sort()
    
    dur_id = list(set(durs))
    dur_id.sort()
    
    Trans_pitch = np.zeros(((len(pitch_id),len(pitch_id),len(pitch_id))))
    Trans_dur = np.zeros(((len(dur_id),len(dur_id),len(dur_id))))
    Trans_vel = np.zeros(((len(vel_id),len(vel_id),len(vel_id))))
                         
    for i in range(len(notes)-2):
        if pitches[i] in pitch_id and pitches[i+1] in pitch_id and pitches[i+2] in pitch_id:
            p1 = pitch_id.index(pitches[i])
            p2 = pitch_id.index(pitches[i+1])
            p3 = pitch_id.index(pitches[i+2])
            Trans_pitch[p1][p2][p3]= Trans_pitch[p1][p2][p3] + 1
        if durs[i] in dur_id and durs[i+1] in dur_id and durs[i+2] in dur_id:
            d1 = dur_id.index(durs[i])
            d2 = dur_id.index(durs[i+1])
            d3 = dur_id.index(durs[i+2])
            Trans_dur[d1,d2,d3] = Trans_dur[d1,d2,d3] + 1
        if velocities[i] in vel_id and velocities[i+1] in vel_id and velocities[i+2] in vel_id:
            v1 = vel_id.index(velocities[i])
            v2 = vel_id.index(velocities[i+1])
            v3 = vel_id.index(velocities[i+2])
            Trans_vel[v1,v2,v3] = Trans_vel[v1,v1,v3] + 1
    
    
    # normalization -- sums of probabilities along the z-axis equals to 0
    z_sums = Trans_pitch.sum(axis=2)

    for i in range(len(pitch_id)):
        for j in range(len(pitch_id)):
            for k in range(len(pitch_id)):
                Trans_pitch[i][j][k] = Trans_pitch[i][j][k]/z_sums[i][j]
                
    
    z_sums = Trans_dur.sum(axis=2)
    
    for i in range(len(dur_id)):
        for j in range(len(dur_id)):
            for k in range(len(dur_id)):
                Trans_dur[i][j][k] = Trans_dur[i][j][k]/z_sums[i][j]
    
    z_sums = Trans_vel.sum(axis=2)
    
    for i in range(len(vel_id)):
        for j in range(len(vel_id)):
            for k in range(len(vel_id)):
                Trans_vel[i][j][k] = Trans_vel[i][j][k]/z_sums[i][j]

    return Trans_pitch, Trans_dur, Trans_vel, dur_id, pitch_id, vel_id

### Training

In [ ]:
file2 = 'Data/Metadata.csv'
user = pd.read_csv(file2, sep=',', encoding='latin-1',skiprows=0)
temp=[]

for row in user.iterrows():
    index, data = row
    for i in ["happy", "sad","calm","angry"]:
        if i in str(data):
            temp.append(data.tolist())
            temp.append(i)
            break
print(temp)

ls0 = []
ls1 = []
darklist = []

vel_d = {}
vel_d2 = {}
dur_d = {}
pitch_d = {}
dur_d2 = {}
pitch_d2 = {}

for i in range(0,len(temp),2):
    ls0.append(temp[i][0])
    ls1.append(temp[i+1])
    
for i in range(len(ls0)):
    string = "Data/MIDI/" +str(ls0[i])+".mid"
    Trans_pitch, Trans_dur, Trans_vel, dur_id, pitch_id, vel_id = markov(string)
    Trans_pitch2, Trans_dur2, Trans_vel2, dur_id2, pitch_id2, vel_id2 = markov2(string)
    
    #creates dictionary containing labels as keys and respective lists of matrixes as values
    if Trans_pitch != [] and Trans_dur != [] and Trans_pitch2 != [] and Trans_dur2 != [] and Trans_vel != [] and Trans_vel2 != []:
        #for first order markov chain
        dur_d[ls1[i]] = dur_d.get(ls1[i], []) + [[Trans_dur, dur_id]]
        pitch_d[ls1[i]] = pitch_d.get(ls1[i], [])+ [[Trans_pitch, pitch_id]]
        vel_d[ls1[i]] = vel_d.get(ls1[i],[]) + [[Trans_vel, vel_id]]
        
        #for second order markov chain
        dur_d2[ls1[i]] = dur_d2.get(ls1[i], []) + [[Trans_dur2, dur_id2]]
        pitch_d2[ls1[i]] = pitch_d2.get(ls1[i], [])+ [[Trans_pitch2, pitch_id2]]
        vel_d2[ls1[i]] = vel_d2.get(ls1[i],[]) + [[Trans_vel2, vel_id2]]
    
    else:
        darklist.append(ls0[i])

### Generation With First Order Markov Chain

In [7]:
def generate_style(style, N, dur_d, pitch_d, vel_d):
    #random combination of duration, velocity, and pitch from different pieces of the same style
    dur_pair = random.choice(dur_d[style])
    Trans_dur = dur_pair[0]
    dur_id = dur_pair[1]
   
    pitch_pair = random.choice(pitch_d[style])
    Trans_pitch = pitch_pair[0]
    pitch_id = pitch_pair[1]
    
    vel_pair = random.choice(vel_d[style])
    Trans_vel = vel_pair[0]
    vel_id = vel_pair[1]
    
    # empty note matrix
    p_i = pitch_id.index(random.choice(pitch_id)); d_i = dur_id.index(random.choice(dur_id)); v_i = vel_id.index(random.choice(vel_id))
    notes2 = np.zeros((N,4))
    notes2[0,[0,1,2,3]] = [0, dur_id[d_i], pitch_id[p_i],vel_id[v_i]]
    
    # roll the dice to generate the rest of the piece
    # arguments taken are the list of choices and their corresponding probabilities
    for i in range(1,N):
        p_i = np.random.choice(np.arange(len(pitch_id)), p = Trans_pitch[p_i,:])
        d_i = np.random.choice(np.arange(len(dur_id)), p = Trans_dur[d_i,:])
        v_i = np.random.choice(np.arange(len(vel_id)), p = Trans_vel[v_i,:])
        offset = notes2[i-1,1]
        notes2[i,[0,1,2,3]] = [offset, offset + dur_id[d_i], pitch_id[p_i],vel_id[v_i]]
    
    notes2[:,[0,1]] = notes2[:,[0,1]]
    notes2 = matrix2notes(notes2)
    return notes2

### Generation With Second Order Markov Chain

In [8]:
def generate_style2(style, N, dur_d2, pitch_d2, vel_d2):
    #random combination of duration, velocity, and pitch from different pieces of the same style
    dur_pair = random.choice(dur_d2[style])
    Trans_dur = dur_pair[0]
    dur_id = dur_pair[1]
    
    pitch_pair = random.choice(pitch_d2[style])
    Trans_pitch = pitch_pair[0]
    pitch_id = pitch_pair[1]
    
    vel_pair = random.choice(vel_d2[style])
    Trans_vel = vel_pair[0]
    vel_id = vel_pair[1]
    
    # empty note matrix
    p_i = pitch_id.index(random.choice(pitch_id)) ; p_i2 = pitch_id.index(random.choice(pitch_id)); d_i = dur_id.index(random.choice(dur_id)); d_i2 = dur_id.index(random.choice(dur_id))
    v_i = vel_id.index(random.choice(vel_id)) ; v_i2 = vel_id.index(random.choice(vel_id))
    notes2 = np.zeros((N,4))
    notes2[0,[0,1,2,3]] = [0, dur_id[d_i], pitch_id[p_i],vel_id[v_i]]
    notes2[1,[0,1,2,3]] = [0, dur_id[d_i2], pitch_id[p_i2],vel_id[v_i2]]
    m = len(pitch_id)
    n = len(dur_id)
    k = len(vel_id)
    
    # roll the dice to generate the rest of the piece
    # arguments taken are the list of choices and their corresponding probabilities
    for i in range(2,N-1):
        p_i3 = np.random.choice(np.arange(m), p = Trans_pitch[p_i2][p_i,:])
        p_i4 = np.random.choice(np.arange(m), p = Trans_pitch[p_i3][p_i2,:])
        
        d_i3 = np.random.choice(np.arange(n), p = Trans_dur[d_i2][d_i,:])
        d_i4 = np.random.choice(np.arange(n), p = Trans_dur[d_i3][d_i2,:])
        
        v_i3 = np.random.choice(np.arange(k), p = Trans_vel[v_i2][v_i,:])
        v_i4 = np.random.choice(np.arange(k), p = Trans_vel[v_i3][v_i2,:])
       
        offset = notes2[i-1,1]
        notes2[i,[0,1,2,3]] = [offset, offset + dur_id[d_i2], pitch_id[p_i2],vel_id[v_i2]]
        offset = notes2[i,1]
        notes2[i+1,[0,1,2,3]] = [offset, offset + dur_id[d_i3], pitch_id[p_i3],vel_id[v_i3]]
        
        p_i, p_i2 = p_i3, p_i4
        d_i, d_i2 = d_i3, d_i4
        v_i, v_i2 = v_i3, v_i4
        
    notes2[:,[0,1]] = notes2[:,[0,1]]
    notes2 = matrix2notes(notes2)
    return notes2

### Generation Test

In [ ]:
voice_music = pretty_midi.PrettyMIDI()

voice_program = pretty_midi.instrument_name_to_program('Voice Oohs')
voice = pretty_midi.Instrument(program=voice_program)

voice.notes = generate_style("angry", 100, dur_d, pitch_d, vel_d) #can change mood as well as order of markov chain used
voice_music.instruments = []
voice_music.instruments.append(voice)
voice_music.write("Test.mid")

#All the mood-based music in our composition was generated in a similar way to this test

## Composition (aside from the parts that used mood-based generation)

### The Dialogue

In [ ]:
#怎么又没电了？
#play(conc(boy(tone3),boy(tone3_),boy(tone4),boy(tone3_),boy(tone4),boy(tone3_)))
librosa.output.write_wav("voice1.wav",conc(boy(tone3),boy(tone3_),boy(tone4),boy(tone3_),boy(tone4),boy(tone3_)),22050)

#嗯？
#play(boy(tone3_, T = .5))
librosa.output.write_wav("voice2.wav",boy(tone3_, T = .5),22050)

#哇！好神奇！
#play(conc(boy(tone4, T = .5),boy(tone3),boy(tone3_),boy(tone3_)))
librosa.output.write_wav("voice3.wav",conc(boy(tone4, T = .5),boy(tone3),boy(tone3_),boy(tone3_)),22050)

#你还能做什么？
#play(conc(boy(tone3),boy(tone3_),boy(tone2),boy(tone4),boy(tone3_),boy(tone1)))
librosa.output.write_wav("voice4.wav",conc(boy(tone3),boy(tone3_),boy(tone2),boy(tone4),boy(tone3_),boy(tone1)),22050)

#干什么呢！不好好学习！罚你一个月不许出去玩儿！
#play(conc(woman(tone4),woman(tone2),woman(tone2_),woman(tone4, T=.5),woman(tone4),woman(tone3),woman(tone3),woman(tone2),woman(tone2,T=.5),woman(tone2),woman(tone3),woman(tone3_),woman(tone2_),woman(tone4),woman(tone4),woman(tone3),woman(tone1),woman(tone4),woman(tone2)))
librosa.output.write_wav('voice5.wav',conc(woman(tone4),woman(tone2),woman(tone2_),woman(tone4, T=.5),woman(tone4),woman(tone3),woman(tone3),woman(tone2),woman(tone2,T=.5),woman(tone2),woman(tone3),woman(tone3_),woman(tone2_),woman(tone4),woman(tone4),woman(tone3),woman(tone1),woman(tone4),woman(tone2)),22050)

#哎，怎么会这样。。。。
#play(conc(boy(tone4,T=.7),boy(tone3, T=.2),boy(tone3_,T=.2),boy(tone2,T=.3),boy(tone4),boy(tone4, T=.5)))
librosa.output.write_wav("voice6.wav",conc(boy(tone4,T=.7),boy(tone3, T=.2),boy(tone3_,T=.2),boy(tone2,T=.3),boy(tone4),boy(tone4, T=.5)),22050)

#你在嘲笑我吗？都怪你！都怪你！
#play(conc(boy(tone3),boy(tone4),boy(tone2),boy(tone4,T=.2),boy(tone3),boy(tone2_, T = .5), boy(tone1),boy(tone4),boy(tone3_,T =.5), boy(tone1),boy(tone4),boy(tone3_,T =.5)))
librosa.output.write_wav("voice7.wav",conc(boy(tone3),boy(tone4),boy(tone2),boy(tone4,T=.2),boy(tone3),boy(tone2_, T = .5), boy(tone1),boy(tone4),boy(tone3_,T =.5), boy(tone1),boy(tone4),boy(tone3_,T =.5)),22050)

#昨天玩儿的很开心，以后有机会再约哦！"
#play(conc(girl(tone2),girl(tone1),girl(tone2),girl(tone4,T=.2),girl(tone3,T=.4),girl(tone1),girl(tone1, T=.5), girl(tone3), girl(tone4,T=.4), girl(tone3), girl(tone1), girl(tone4,T=.2), girl(tone4), girl(tone1), girl(tone2_, T=.5)))
librosa.output.write_wav("voice8.wav",conc(girl(tone2),girl(tone1),girl(tone2),girl(tone4,T=.2),girl(tone3,T=.4),girl(tone1),girl(tone1, T=.5), girl(tone3), girl(tone4,T=.4), girl(tone3), girl(tone1), girl(tone4,T=.2), girl(tone4), girl(tone1), girl(tone2_, T=.5)),22050)

#yes!太好了！太好了！
#play(conc(boy(tone4, T=.5), boy(tone4), boy(tone3), boy(tone3_), boy(tone4), boy(tone3), boy(tone3_, T=.5)))
librosa.output.write_wav("voice9.wav",conc(boy(tone4, T=.5), boy(tone4), boy(tone3), boy(tone3_), boy(tone4), boy(tone3), boy(tone3_, T=.5)),22050)

#哎，人生，太美好了
#play(conc(boy(tone4, T=.5), boy(tone2), boy(tone1, T=.5), boy(tone4), boy(tone2), boy(tone3), boy(tone2_, T=.5)))
librosa.output.write_wav("voice10.wav",conc(boy(tone4, T=.5), boy(tone2), boy(tone1, T=.5), boy(tone4), boy(tone2), boy(tone3), boy(tone2_, T=.5)),22050)

#啊对你还有什么功--
#play(conc(boy(tone1), boy(tone4, T=.5), boy(tone3), boy(tone2), boy(tone3), boy(tone3_), boy(tone2_), boy(tone1, T=.1)))
librosa.output.write_wav("voice11.wav",conc(boy(tone1), boy(tone4, T=.5), boy(tone3), boy(tone2), boy(tone3), boy(tone3_), boy(tone2_), boy(tone1, T=.1)),22050)

### The iPhone Sounds

In [9]:
def charge(f,T=1.):
    return [sin(8*pi*f*n + (4*n) * sin(4*pi*f*n)) * exp(-5*n) for n in arange(0.,T,1./44100)]

In [ ]:
#the remix
play(conc(charge(440,T=.5),charge(440,T=.5),charge(440,T=.5),charge(440,T=.125),charge(466,T=.125),charge(440,T=.125),charge(466,T=.125),charge(440,T=.5),charge(440,T=.5),charge(440,T=.5),charge(440,T=.125),charge(466,T=.125),charge(523,T=.125),charge(466,T=.125),charge(440,T=.5),charge(440,T=.5),charge(440,T=.5),charge(440,T=.125),charge(392,T=.125),charge(349,T=.125),charge(329,T=.125),charge(294,T=.5),charge(294,T=.5),charge(294)))

In [ ]:
#the ringtone
apple = pretty_midi.PrettyMIDI()
p = pretty_midi.instrument_name_to_program("Marimba")
marimba = pretty_midi.Instrument(p)

note_names1 = ["C3","Bb3","A3","Eb3","C3","E3","F3","C3","Bb3","A3","Eb3","C3","E3","F3"]
starts1 = [0,0.75,1.75,2.75,3.25,3.5,3.75,4,4.75,5.75,6.75,7.25,7.5,7.75]
ends1 = [0.75,1.75,2,3.25,3.5,3.75,4,4.75,5.75,6,7.25,7.5,7.75,8]

note_names2 = ["C5","Bb4","G4","C5","F4","C5","Bb4","C5","F4","G4","G4","Bb4","C5","C5","Bb4","G4","C5","F4","C5","Bb4","C5","F4","G4","G4","Bb4","C5"]
starts2 = [0,0.125,0.25,0.5,0.75,1,1.25,1.5,1.75,2.75,3.25,3.5,3.75,4,4.125,4.25,4.5,4.75,5,5.25,5.5,5.75,6.75,7.25,7.5,7.75]
ends2 = [0.125,0.25,0.5,0.75,1,1.25,1.5,1.75,2,3.25,3.5,3.75,4,4.125,4.25,4.5,4.75,5,5.25,5.5,5.75,6,7.25,7.5,7.75,8]

note_names = note_names1 + note_names2
starts = starts1 + starts2
ends = ends1 + ends2

notes = []
for i in note_names:
    notes.append(pretty_midi.note_name_to_number(i))

marimba.notes = []
apple.instruments = []
marimba.notes = matrix2notes(create_note_matrix(np.array(starts),np.array(ends),notes,100))
apple.instruments.append(marimba)
play(apple.synthesize(44100))

### Other Sounds

In [11]:
#the claps and footsteps and door knocks
music = pretty_midi.PrettyMIDI()
program = pretty_midi.instrument_name_to_program("Applause")
a = pretty_midi.Instrument(program,is_drum = True)
a.notes = []

start = .5
end =.9
for i in range(15):
    a.notes.append(pretty_midi.Note(100,39,start,end))
    start += 1
    end = start + 0.4
    
music.instruments = []
music.instruments.append(a)

music.write("clap.mid")

In [ ]:
#the bit-wise BGM
bgm = [abs((t*math.pow(1,(0x75054970>>((t>>13&7)<<2)&0xF)/12+2)*((t>>11&3)+1))%256-128)-64 for t in range(64000 * 4)]
plt.plot(bgm,'.')
plt.show()
IPython.display.Audio(data=bgm, rate=8000) # press the "play" button to hear audio

In [ ]:
#the buzzing sound in the remix part
Fm = 886; index = 100
gmin2 =  [cos(2*pi*440*n + index * cos(2*pi*Fm*n))*(exp(-5*n)) for n in arange(0.,2,1./44100)]
gmin2 = np.array(gmin2, np.double)
IPython.display.Audio(data=gmin2, rate=44100) # press the "play" button to hear audio